In [6]:
from pyquery import PyQuery as pq
from tqdm import tqdm

import csv
import datetime
from glob import glob
import sys

In [7]:
rssdir = "rss/"
masterlist = []
for rssfile in tqdm(glob(rssdir + "*")):
    if ".aria2" not in rssfile:   # Skip stuff still downloading
        indexfile = rssfile.replace(rssdir, "").replace(rssdir.replace('/', '\\'), '')
        with open(rssfile, "rb") as infile:
            html = infile.read()
            for item in pq(html)("item"):
                line = {}
                line["rssindex"] = indexfile
                authors = []
                for au in pq(item)("dc\:creator"):
                    authors.append(pq(au).text().strip())
                line["authors"] = "|".join(authors)
                line['title'] = pq(pq(item)("title")).text().strip()
                line['introtext'] = pq(pq(pq(item)("description"))[0]).html()
                line['good-date'] = ""
                line['original-date'] = pq(pq(item).html().replace("pubDate", "pubdate"))("pubdate").text()
                # Tue, 25 Aug 2020 11:38:02 GMT
                line["good-date"] = datetime.datetime.strptime(line['original-date'], "%a, %d %b %Y %H:%M:%S +0000")
                line["good-date"] = datetime.datetime.strftime(line['good-date'], "%Y-%m-%d %H%M%S")    
                line["maybe-archive"] = "https://web.archive.org/web/*/"
                line["maybe-archive"] += pq(pq(item)("link")).text().replace("https://cms.", "").replace("https://www.", "")    
                line['original-link'] = pq(pq(item)("link")).text()
                categories = []
                for cat in pq(item)("category"):
                    categories.append(pq(cat).text())
                line['categories'] = '|'.join(categories)
                masterlist.append(line)

100%|██████████████████████████████████████████████████████████████████████████████| 6625/6625 [02:15<00:00, 49.07it/s]


In [10]:
masterlist = sorted(masterlist, key=lambda d: d['good-date'], reverse=True)


In [11]:
with open("dcist-rss-index.csv", "w", newline="", encoding="utf-8") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(list(masterlist[0].keys()))
    for row in sorted(masterlist, key=lambda d: d['good-date'], reverse=True):
        writer.writerow(list(row.values()))

In [13]:
authorindex = {}
for row in masterlist:
    for author in row['authors'].split("|"):
        if author not in authorindex:
            authorindex[author] = 0
        authorindex[author] += 1

In [14]:
with open("author-index.csv", "w", newline="", encoding="utf-8") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(["author", "storycount"])
    for author in sorted(authorindex):
        storycount = authorindex[author]
        writer.writerow([author, storycount])